In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import torch

Define dir to data pickles

In [2]:
data_path = "data"

era_5_evap_dir = os.path.join(data_path,"era5_evaporation.pickle")
era_5_soil_lvl1_dir = os.path.join(data_path, "era5_vol_soil_lvl_1.pickle")
era_5_soil_lvl2_dir = os.path.join(data_path, "era5_vol_soil_lvl_2.pickle")
era_5_soil_lvl3_dir = os.path.join(data_path, "era5_vol_soil_lvl_3.pickle")
era_5_soil_lvl4_dir = os.path.join(data_path, "era5_vol_soil_lvl_4.pickle")
precip_dir = os.path.join(data_path, "gpm-imerg_df.pickle")
grace_dir = os.path.join(data_path, "grace_df.pickle")
target_dir = os.path.join(data_path, "II_113_1.pickle")

In [3]:
def read_and_preprocess_data(data_dir, prefix):
    pickle = pd.read_pickle(data_dir)
    pickle["value"] = pickle["value"].apply(lambda x: np.nan_to_num(x=x, nan=np.nanmean(x)))
    pickle["mask"] = pickle["mask"].apply(lambda x: x.astype(float))
    pickle["value"] = pickle.apply(lambda x: np.array([x["value"], x["mask"]]), axis=1)
    pickle = pickle.drop(columns=['mask'])
    pickle = pickle.rename(columns={"value": prefix + "_value"})
    pickle = pickle.sort_values(by="date", ignore_index=True)
    pickle = pickle.set_index("date")
    return pickle

In [4]:
def read_and_preprocess_data(data_dir, prefix):
    pickle = pd.read_pickle(data_dir)
    pickle["value"] = pickle["value"].apply(lambda x: np.nan_to_num(x=x, nan=np.nanmean(x)))
    pickle["mask"] = pickle["mask"].apply(lambda x: x.astype(float))
    pickle = pickle.rename(columns={"value": prefix + "_value", "mask": prefix + "_mask"})
    pickle = pickle.sort_values(by="date", ignore_index=True)
    pickle = pickle.set_index("date")
    return pickle

In [5]:
def read_soil_lvls(lvl1_dir, lvl2_dir, lvl3_dir, lvl4_dir):
    return read_and_preprocess_data(lvl1_dir, "lvl1"), read_and_preprocess_data(lvl2_dir, "lvl2"), read_and_preprocess_data(lvl3_dir, "lvl3"), read_and_preprocess_data(lvl4_dir, "lvl4")

In [6]:
evaporation_data = read_and_preprocess_data(era_5_evap_dir, "evap")

In [7]:
lvl1, lvl2, lvl3, lvl4 = read_soil_lvls(era_5_soil_lvl1_dir, era_5_soil_lvl2_dir, era_5_soil_lvl3_dir, era_5_soil_lvl4_dir)

In [8]:
precip_data = read_and_preprocess_data(precip_dir, "precip")

In [9]:
precip_data.describe()

,precip_value,precip_mask
count,244,244
unique,244,244
top,"[50.97333, 50.97333, 50.97333, 50.97333, 50.97...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
freq,1,1


In [10]:
precip_data.head()

,precip_value,precip_mask
date,,
2002-01-01,"[50.97333, 50.97333, 50.97333, 50.97333, 50.97...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2002-02-01,"[85.69334, 85.69334, 85.69334, 85.69334, 85.69...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2002-03-01,"[10.026667, 10.026667, 10.026667, 10.026667, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2002-04-01,"[21.533333, 21.533333, 21.533333, 21.533333, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
2002-05-01,"[246.52, 246.52, 246.52, 246.52, 246.52, 246.5...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."


In [11]:
grace_data = pd.read_pickle(grace_dir)
grace_data = grace_data.rename(columns = {"value": "grace_value"})
grace_data = grace_data.sort_values(by="date", ignore_index=True)
print(type(grace_data["date"][0]))

grace_data = grace_data.set_index("date")

grace_data = read_and_preprocess_data(grace_dir, "grace")

<class 'str'>


In [12]:
grace_data.tail()

,grace_value,grace_mask
date,,
2021-10-01,"[-12.818175138233492, -12.818175138233492, -12...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-11-01,"[-11.360681991513632, -11.360681991513632, -11...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2021-12-01,"[-7.66136392423689, -7.66136392423689, -7.6613...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2022-01-01,"[-5.594853680047669, -5.594853680047669, -5.59...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2022-02-01,"[-3.360284155330798, -3.360284155330798, -3.36...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
target = pd.read_pickle(target_dir)
target = target.rename(columns = {"value": "target"})
target = target.sort_values(by="date", ignore_index=True)
target = target.set_index("date")
target = read_and_preprocess_data(target_dir, "target")

In [14]:
dfs = [evaporation_data, lvl1, lvl2, lvl3, lvl4, precip_data, grace_data, target]

In [15]:
data = pd.concat(dfs, axis = 1)

In [16]:
data[["evap_value", "lvl1_value", "lvl2_value", "lvl3_value", "lvl4_value"]]

,evap_value,lvl1_value,lvl2_value,lvl3_value,lvl4_value
date,,,,,
2002-01-01,"[-0.00027116016, -0.00027116016, -0.0002358034...","[0.25207317, 0.25207317, 0.25037944, 0.2503794...","[0.25416452, 0.25416452, 0.25248224, 0.2524822...","[0.25356066, 0.25356066, 0.25290644, 0.2529064...","[0.2390303, 0.2390303, 0.24438423, 0.24438423,..."
2002-02-01,"[-0.00087652216, -0.00087652216, -0.0008312897...","[0.2616769, 0.2616769, 0.26138318, 0.26138318,...","[0.2636571, 0.2636571, 0.26317835, 0.26317835,...","[0.27171874, 0.27171874, 0.27298903, 0.2729890...","[0.25494355, 0.25494355, 0.2600705, 0.2600705,..."
2002-03-01,"[-0.0013092221, -0.0013092221, -0.0012750574, ...","[0.23586917, 0.23586917, 0.23601604, 0.2360160...","[0.23867369, 0.23867369, 0.23849058, 0.2384905...","[0.25597513, 0.25597513, 0.25510728, 0.2551072...","[0.26234245, 0.26234245, 0.26519203, 0.2651920..."
2002-04-01,"[-0.0018067344, -0.0018067344, -0.0017491414, ...","[0.17545515, 0.17545515, 0.17230803, 0.1723080...","[0.16594648, 0.16594648, 0.1648364, 0.1648364,...","[0.18871498, 0.18871498, 0.18579292, 0.1857929...","[0.2574166, 0.2574166, 0.2593869, 0.2593869, 0..."
2002-05-01,"[-0.002586316, -0.002586316, -0.0024680307, -0...","[0.14691228, 0.14691228, 0.14593571, 0.1459357...","[0.092069924, 0.092069924, 0.084379494, 0.0843...","[0.10689336, 0.10689336, 0.105703175, 0.105703...","[0.2441786, 0.2441786, 0.24538213, 0.24538213,..."
...,...,...,...,...,...
2022-08-01,"[-0.0028839423, -0.0028839423, -0.00297212, -0...","[0.2136339, 0.2136339, 0.22499788, 0.22499788,...","[0.19729424, 0.19729424, 0.21057701, 0.2105770...","[0.08790189, 0.08790189, 0.11650449, 0.1165044...","[0.17921698, 0.17921698, 0.19504035, 0.1950403..."
2022-09-01,"[-0.001634782, -0.001634782, -0.0016639845, -0...","[0.23106688, 0.23106688, 0.23695487, 0.2369548...","[0.22172177, 0.22172177, 0.23043644, 0.2304364...","[0.11823386, 0.11823386, 0.15069693, 0.1506969...","[0.17700422, 0.17700422, 0.19253767, 0.1925376..."
2022-10-01,"[-0.0009947373, -0.0009947373, -0.0009996509, ...","[0.21744365, 0.21744365, 0.2255041, 0.2255041,...","[0.21135658, 0.21135658, 0.22264045, 0.2226404...","[0.17355758, 0.17355758, 0.20202285, 0.2020228...","[0.17668408, 0.17668408, 0.19297284, 0.1929728..."


In [17]:
mask = data['target_value'].isna()

# stwórz serię liczb całkowitych, które będą używane do stworzenia grup
groups = (mask != mask.shift()).cumsum()

# stwórz grupy, dla których w kolumnie nie ma wartości NaN
valid_groups = groups[~mask]

# policz długości grup
group_lengths = valid_groups.groupby(valid_groups).size()

# znajdź indeks grupy o największej długości
longest_group_index = group_lengths.idxmax()

# stwórz maskę dla wierszy należących do grupy o największej długości
longest_group_mask = (groups == longest_group_index) & (~mask)

# wybierz wiersze należące do grupy o największej długości
longest_group = data[longest_group_mask]


In [18]:
data = data.iloc[:239]

In [19]:
data.isna().sum()

evap_value       0
evap_mask        0
lvl1_value       0
lvl1_mask        0
lvl2_value       0
lvl2_mask        0
lvl3_value       0
lvl3_mask        0
lvl4_value       0
lvl4_mask        0
precip_value     0
precip_mask      0
grace_value     36
grace_mask      36
target_value     0
target_mask      0
dtype: int64

In [20]:
nan = data.loc[data["grace_value"].isna()]

In [21]:
def fill_missing_value(nans, idx):
    l = []
    for i in range(nans):
        arr = np.zeros(104)
        arr[0] = -9999999
        l.append(arr)
    l = pd.Series(l, idx)
    return l

In [22]:
def fill_missing_nans(nans, idx):
    l = []
    for i in range(nans):
        arr = np.ones(104, dtype=np.float32)
        arr[0] = 0.0
        l.append(arr)
    l = pd.Series(l, idx)
    return l

In [23]:
idx = data['grace_value'].loc[data['grace_value'].isna()].index 
data['grace_value'].loc[data['grace_value'].isna()] = fill_missing_value(36, idx)

In [24]:
data["grace_mask"].loc[data['grace_mask'].isna()] = fill_missing_nans(36, idx)

In [1]:
data.head()

NameError: name 'data' is not defined

In [35]:
idxs = data.index
target_n_1 = []
for index in range(len(data)):
    if index == 0:
        arr = np.zeros(104)
        arr[0] = -9999999
        target_n_1.append(arr)
    else:
        target_n_1.append(data.iloc[index-1]["target_value"])
        
target_n_1 = pd.Series(target_n_1, idxs)

In [36]:
data = data.assign(target_n_1 = target_n_1)

In [38]:
data.head()

,evap_value,evap_mask,lvl1_value,lvl1_mask,lvl2_value,lvl2_mask,lvl3_value,lvl3_mask,lvl4_value,lvl4_mask,precip_value,precip_mask,grace_value,grace_mask,target_value,target_mask,target_n_1
date,,,,,,,,,,,,,,,,,
2002-01-01,"[-0.00027116016, -0.00027116016, -0.0002358034...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.25207317, 0.25207317, 0.25037944, 0.2503794...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.25416452, 0.25416452, 0.25248224, 0.2524822...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.25356066, 0.25356066, 0.25290644, 0.2529064...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2390303, 0.2390303, 0.24438423, 0.24438423,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[50.97333, 50.97333, 50.97333, 50.97333, 50.97...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[-9999999.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.939999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[-9999999.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2002-02-01,"[-0.00087652216, -0.00087652216, -0.0008312897...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2616769, 0.2616769, 0.26138318, 0.26138318,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2636571, 0.2636571, 0.26317835, 0.26317835,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.27171874, 0.27171874, 0.27298903, 0.2729890...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.25494355, 0.25494355, 0.2600705, 0.2600705,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[85.69334, 85.69334, 85.69334, 85.69334, 85.69...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[-9999999.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.810000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.939999999999998, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2002-03-01,"[-0.0013092221, -0.0013092221, -0.0012750574, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.23586917, 0.23586917, 0.23601604, 0.2360160...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.23867369, 0.23867369, 0.23849058, 0.2384905...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.25597513, 0.25597513, 0.25510728, 0.2551072...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.26234245, 0.26234245, 0.26519203, 0.2651920...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[10.026667, 10.026667, 10.026667, 10.026667, 1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[-9999999.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.810000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2002-04-01,"[-0.0018067344, -0.0018067344, -0.0017491414, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.17545515, 0.17545515, 0.17230803, 0.1723080...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.16594648, 0.16594648, 0.1648364, 0.1648364,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.18871498, 0.18871498, 0.18579292, 0.1857929...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.2574166, 0.2574166, 0.2593869, 0.2593869, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[21.533333, 21.533333, 21.533333, 21.533333, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[4.137165727263286, 4.137165727263286, 4.13716...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[31.82, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[31.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2002-05-01,"[-0.002586316, -0.002586316, -0.0024680307, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.14691228, 0.14691228, 0.14593571, 0.1459357...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [221]:
tensor = torch.tensor(data.iloc[0].tolist())

In [66]:
np.array(data.iloc[0].tolist(), dtype="float32")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (14,) + inhomogeneous part.

In [71]:
data.iloc[0].apply(lambda row: print(len(row)))

18
18
18
18
18
18
18
18
18
18
104
104


TypeError: object of type 'int' has no len()

In [65]:
data.to_pickle("data/ready_dataset.pickle")

In [44]:
torch.tensor(data.iloc[0].values, dtype=torch.float32)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [50]:
torch.tensor(np.stack( data.iloc[0].values, axis=0 ), dtype=torch.float32)

tensor([[-2.7116e-04, -2.7116e-04, -2.3580e-04,  ..., -2.1271e-04,
         -2.3671e-04, -2.3671e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          1.0000e+00,  1.0000e+00],
        [ 2.5207e-01,  2.5207e-01,  2.5038e-01,  ...,  3.4124e-01,
          2.6794e-01,  2.6794e-01],
        ...,
        [ 3.1940e+01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
          1.0000e+00,  1.0000e+00],
        [-1.0000e+07,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

In [64]:
data.loc[0, data.columns != "target_value"].values

array([array([-0.00027116, -0.00027116, -0.0002358 , -0.0002358 , -0.00022406,
              -0.00022406, -0.00023284, -0.00023284, -0.00023284, -0.00024393,
              -0.00024393, -0.00025166, -0.00025166, -0.00027116, -0.00027116,
              -0.0002358 , -0.0002358 , -0.00022406, -0.00022406, -0.00023284,
              -0.00023284, -0.00023284, -0.00024393, -0.00024393, -0.00025166,
              -0.00025166, -0.00027116, -0.00027116, -0.0002358 , -0.0002358 ,
              -0.00022406, -0.00022406, -0.00023284, -0.00023284, -0.00023284,
              -0.00024393, -0.00024393, -0.00025166, -0.00025166, -0.00028515,
              -0.00028515, -0.00024109, -0.00024109, -0.00021623, -0.00021623,
              -0.00021149, -0.00021149, -0.00021149, -0.0002333 , -0.0002333 ,
              -0.00026857, -0.00026857, -0.00028515, -0.00028515, -0.00024109,
              -0.00024109, -0.00021623, -0.00021623, -0.00021149, -0.00021149,
              -0.00021149, -0.0002333 , -0.0002333 ,

In [63]:
#data = data.reset_index()
data = data.drop(columns=["date"])

In [ ]:
data.to_pickle("